# Trainer
千帆Python SDK 在使用[resource API实现发起训练微调](./console-finetune.ipynb)之外，还提供了Trainer API，可以更方便地实现一体化的训练微调pipeline。同时提供了状态事件回调函数的注册，通过事件分发实现训练流程状态事件的监控。


本例将基于qianfan==0.2.1展示通过Dataset加载本地数据集，并上传到千帆平台，基于LLama-2-7b进行fine-tune，直到最终完成服务发布，并最终实现服务调用的完整过程。

## 前置准备
- 初始化千帆安全认证AK、SK

In [ ]:
import os 

os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

## 数据集加载

千帆SDK提供了数据集实现帮助我们可以快速的加载本地的数据集到内存，并通过设定DataSource数据源以保存至本地和千帆平台。

In [ ]:

from qianfan.dataset import Dataset
from qianfan.trainer import LLMFinetune

# 加载本地数据集
ds: Dataset = Dataset.load(data_file="./xx.jsonl")

ds.save()

In [ ]:
# 对于需要在训练过程中监控每个阶段的各个节点的用户，可以通过事件回调函数来实现
from qianfan.trainer.event import Event, EventHandler
from qianfan.trainer.consts import ActionState
from qianfan.resources.console import consts as console_consts
from qianfan.trainer.configs import TrainConfig
from qianfan.trainer.base import Pipeline
from qianfan.trainer.model import Model, Service, DeployConfig
from typing import cast

# 定义自己的EventHandler，并实现dispatch方法
class InferAfterSFT(EventHandler):
    ppl: Pipeline
    def dispatch(self, event: Event) -> None:
        print("receive:<", event)
        if event.action_state == ActionState.Done:
            svc = cast(Service, event.data["service"])
            
trainer = LLMFinetune(
    train_type="Llama-2-13b",
    dataset=ds,
    train_config=TrainConfig(
        epochs=1,
        batch_size=1,
        learning_rate=0.00002,
    ),
    deploy_config=DeployConfig(
        name="fin_llama213b",
        replicas=1,
        pool_type=console_consts.DeployPoolType.PrivateResource,
    ),
)
eh = InferAfterSFT(trainer.ppls[0])
trainer.register_event_handler(eh)
trainer.run()



In [ ]:
# 部署模型服务，使用Model，Service调用模型服务
from qianfan.trainer import Model, Service
from qianfan.trainer.configs import DeployConfig, TrainConfig
from qianfan.resources.console import consts as console_consts
# 对于自定义需求较强的用户，可以根据不同的模型传入不同的TrainConfig 以指定训练过程中的参数
trainer = LLMFinetune(
    train_type="ChatGLM2-6B",
    dataset=ds,
    train_config=TrainConfig(
        epochs=1, 
        batch_size=1, 
        learning_rate=0.00002, 
    ),
    deploy_config=DeployConfig(
        replicas=1,
        pool_type=console_consts.DeployPoolType.PrivateResource,
    )
)
trainer.run()

res = trainer.result[0][""]

## 自定义pipeline

